In [927]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np

# Send HTTP GET request to website
page = requests.get("https://en.wikipedia.org/wiki/2020_Singaporean_general_election")
# Check whether request was successful
print(page.status_code)

200


In [928]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")
sortable_tables = soup.find_all('table', {'class':'wikitable'})

# Get tables that have the word "Seat" in it
seats = pd.read_html(str(sortable_tables), match='Seat')

In [929]:
ge_results_table = seats[3]
ge_results_table

Candidates and results of 2020 Singaporean general election [195][196]  \
                                                                 Division   
0                                Bukit Batok SMC[197]                       
1                                Bukit Batok SMC[197]                       
2                              Bukit Panjang SMC[198]                       
3                              Bukit Panjang SMC[198]                       
4                             Hong Kah North SMC[199]                       
..                                                ...                       
59                                  Tampines GRC[225]                       
60                             Tanjong Pagar GRC[226]                       
61                             Tanjong Pagar GRC[226]                       
62                                West Coast GRC[227]                       
63                                West Coast GRC[227]                       

                                                      \
   Seat  Voters Rejected                       Party   
0     1   29948      533       People's Action Party   
1     1   29948      533  Singapore Democratic Party   
2     1   35437      586       People's Action Party   
3     1   35437      586  Singapore Democratic Party   
4     1   28046      403       People's Action Party   
..  ...     ...      ...                         ...   
59    5  151589     3521   National Solidarity Party   
60    5  134494     1933       People's Action Party   
61    5  134494     1933    Progress Singapore Party   
62    5  146089     1646       People's Action Party   
63    5  146089     1646    Progress Singapore Party   

                                                                           \
                                         Candidate(s)  Votes      Votes %   
0                                       Murali Pillai  15500  54.80 / 100   
1                                      Chee Soon Juan  12787  45.20 / 100   
2                                       Liang Eng Hwa  18085  53.73 / 100   
3                                        Paul Tambyah  15576  46.27 / 100   
4                                            Amy Khor  16347  60.99 / 100   
..                                                ...    ...          ...   
59  Reno Fong Mohd Ridzwan Mohammad Yeo Ren-Yuan C...  47875  33.59 / 100   
60  Chan Chun Sing Indranee Rajah Joan Pereira Eri...  78330  63.10 / 100   
61  Wendy Low Harish Pillay Michael Chua Teck Leon...  45807  36.90 / 100   
62  S. Iswaran Desmond Lee Foo Mee Har Rachel Ong ...  71658  51.68 / 100   
63  Tan Cheng Bock Leong Mun Wai Hazel Poa Nadaraj...  66996  48.32 / 100   

                                   
   Sample counts %  Swing Margins  
0         57 / 100   6.43   9.60%  
1         43 / 100   6.43   9.60%  
2         56 / 100  14.65   7.46%  
3         44 / 100  14.65   7.46%  
4         63 / 100  13.77  21.98%  
..             ...    ...     ...  
59        33 / 100   5.65  32.82%  
60        63 / 100  14.61  26.20%  
61        37 / 100  14.61  26.20%  
62        52 / 100  26.89   3.36%  
63        48 / 100    NaN   3.36%  

[64 rows x 11 columns]

In [930]:
# Rename the columns to make it easier to handle the data. 'Division' is also renamed to 'Area' and 'Votes %' to 'Percentage' to keep it consistent with the data that was scraped by Mathavan.
ge_results_table.columns = ['Area', 'Seat', 'Voters', 'Rejected', 'Party', 'Candidates', 'Votes', 'Percentage', 'Sample Counts %', 'Swing', 'Margins']

# Cleaning up the Area names to remove the wiki citations and to keep it consistent with data that Mathavan scraped. 
ge_results_table['Area'] = ge_results_table['Area'].apply(lambda x: re.sub('(GRC|SMC)+(\[.*?\])', '', x).rstrip())

# Cleaning up 'Percentage', 'Sample counts %' and 'Margins' in order to convert them to numeric
ge_results_table['Percentage'] = ge_results_table['Percentage'].apply(lambda x: re.sub('/.*$', '', x).strip())
ge_results_table['Sample Counts %'] = ge_results_table['Sample Counts %'].apply(lambda x: re.sub('/.*$', '', x).strip())
ge_results_table['Margins'] = ge_results_table['Margins'].apply(lambda x: re.sub('%', '', x).strip())

# Convert to numeric
ge_results_table['Percentage'] = ge_results_table['Percentage'].astype(float)
ge_results_table['Margins'] = ge_results_table['Margins'].astype(float)
ge_results_table['Sample Counts %'] = ge_results_table['Sample Counts %'].astype(int)

ge_results_table.dtypes

Area                object
Seat                 int64
Voters               int64
Rejected             int64
Party               object
Candidates          object
Votes                int64
Percentage         float64
Sample Counts %      int32
Swing              float64
Margins            float64
dtype: object

In [931]:
# Dropping 'Candidates' column as we won't need it for our analysis later.
# Dropping 'Rejected' column as the previous wiki pages are missing this information. We can get 'Rejected' from Hui Jing's scraped data later.
ge_results_table = ge_results_table.drop('Candidates', axis=1)
ge_results_table = ge_results_table.drop('Rejected', axis=1)

In [932]:
# Adding 'GRC/SMC' columns for analysis later
ge_results_table['GRC/SMC'] = ge_results_table['Seat'].apply(lambda x: 'SMC' if x==1 else 'GRC')
# Adding 'Year' column for combining data later
ge_results_table['Year'] = 2020

ge_results_table

,Area,Seat,Voters,Party,Votes,Percentage,Sample Counts %,Swing,Margins,GRC/SMC,Year
0,Bukit Batok,1,29948,People's Action Party,15500,54.80,57,6.43,9.60,SMC,2020
1,Bukit Batok,1,29948,Singapore Democratic Party,12787,45.20,43,6.43,9.60,SMC,2020
2,Bukit Panjang,1,35437,People's Action Party,18085,53.73,56,14.65,7.46,SMC,2020
3,Bukit Panjang,1,35437,Singapore Democratic Party,15576,46.27,44,14.65,7.46,SMC,2020
4,Hong Kah North,1,28046,People's Action Party,16347,60.99,63,13.77,21.98,SMC,2020
...,...,...,...,...,...,...,...,...,...,...,...
59,Tampines,5,151589,National Solidarity Party,47875,33.59,33,5.65,32.82,GRC,2020
60,Tanjong Pagar,5,134494,People's Action Party,78330,63.10,63,14.61,26.20,GRC,2020
61,Tanjong Pagar,5,134494,Progress Singapore Party,45807,36.90,37,14.61,26.20,GRC,2020
62,West Coast,5,146089,People's Action Party,71658,51.68,52,26.89,3.36,GRC,2020


In [933]:
# Storing electoral_divisions_table
electoral_divisions_table = seats[1]
electoral_divisions_table

,Unnamed: 0,2015,2020
0,Seats,89,93
1,Electoral divisions,29,31
2,Group representation constituencies,16,17
3,Four-Member GRCs,6,6
4,Five-Member GRCs,8,11
5,Six-Member GRCs,2,0
6,Single member constituencies,13,14
7,Voters,2458058,2647372
8,Voters (overseas votes inclusive),2462926,2653942


In [934]:
# Checking next available table
seats[2]

,Party,Party.1,Votes,%,+/–,Seats,+/–.1
0,NaN,People's Action Party,1527491,61.23,–8.63,83,0
1,NaN,Workers' Party,279922,11.22,–1.26,10,+4
2,NaN,Progress Singapore Party,253996,10.18,New,2,New
3,NaN,Singapore Democratic Party,111054,4.45,+0.92,0,0
4,NaN,National Solidarity Party,93653,3.75,+0.22,0,0
5,NaN,Peoples Voice,59183,2.37,New,0,New
6,NaN,Reform Party,54599,2.19,–0.44,0,0
7,NaN,Singapore People's Party,37998,1.52,–0.65,0,0
8,NaN,Singapore Democratic Alliance,37237,1.49,–0.57,0,0
9,NaN,Red Dot United,31260,1.25,New,0,New


In [935]:
# Only keeping the rows for valid_votes, invalid/blank votes, total votes, registered voters/turnout, as the other rows can be gotten through aggregation later.
votes_dict = {'Year': 2020,
              'Registered Voters/Turnout': seats[2]['Votes'].iloc[17],
              'Registered Voters/Turnout %': seats[2]['%'].iloc[17],
              'Total Votes': seats[2]['Votes'].iloc[16], 
              'Total Votes %': seats[2]['%'].iloc[16],
              'Valid Votes': seats[2]['Votes'].iloc[14], 
              'Valid Votes %': seats[2]['%'].iloc[14],
              'Invalid/Blank Votes': seats[2]['Votes'].iloc[15],
              'Invalid/Blank Votes %': seats[2]['%'].iloc[15]
             }

# Create a new votes_table to store this info
votes_table = pd.DataFrame(data = votes_dict, index=[0])
votes_table.set_index('Year', inplace=True)
votes_table

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2020,2651435,95.81,2540359,100.00,2494537,98.20,45822,1.80


In [ ]:
# Send HTTP GET request to website to retrieve 2015's election data
page = requests.get("https://en.wikipedia.org/wiki/2015_Singaporean_general_election")
page.status_code

In [936]:
soup = BeautifulSoup(page.content, "html.parser")
sortable_tables = soup.find_all('table', {'class': 'wikitable'})

# Get tables that have the word "Seat" in it
seats = pd.read_html(str(sortable_tables), match='Seat')

table_tmp = seats[2]
table_tmp

,Party,Party.1,Votes,%,Seats,+/–
0,NaN,People's Action Party,1579183,69.86,83,+2
1,NaN,Workers' Party,282143,12.48,6,0
2,NaN,Singapore Democratic Party,84931,3.76,0,0
3,NaN,National Solidarity Party,79826,3.53,0,0
4,NaN,Reform Party,59517,2.63,0,0
5,NaN,Singaporeans First,50867,2.25,0,New
6,NaN,Singapore People's Party,49107,2.17,0,0
7,NaN,Singapore Democratic Alliance,46550,2.06,0,0
8,NaN,People's Power Party,25475,1.13,0,New
9,NaN,Independents,2780,0.12,0,New


In [937]:
# Storing the rows we want into a new temporary table
votes_dict = {'Year': 2015,
              'Registered Voters/Turnout': table_tmp['Votes'].iloc[15],
              'Registered Voters/Turnout %': table_tmp['%'].iloc[15],
              'Total Votes': table_tmp['Votes'].iloc[14],
              'Total Votes %': table_tmp['%'].iloc[14],
              'Valid Votes': table_tmp['Votes'].iloc[12],
              'Valid Votes %': table_tmp['%'].iloc[12],
              'Invalid/Blank Votes': table_tmp['Votes'].iloc[13],
              'Invalid/Blank Votes %': table_tmp['%'].iloc[13]
             }

votes_table_tmp = pd.DataFrame(data=votes_dict, index = [0])
votes_table_tmp.set_index('Year', inplace=True)
votes_table_tmp

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2015,2462926,93.70,2307746,100.00,2260379,97.95,47367,2.05


In [938]:
# Combining this temporary table to our votes_table
votes_table = pd.concat([votes_table, votes_table_tmp], axis=0)
votes_table

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2020,2651435,95.81,2540359,100.00,2494537,98.20,45822,1.80
2015,2462926,93.70,2307746,100.00,2260379,97.95,47367,2.05


In [939]:
# Handling the next table
table_tmp = seats[3]
table_tmp

Candidates and results of 2015 Singaporean general election [80]        \
                                                           Division Seats   
0                                     Bukit Batok SMC                   1   
1                                     Bukit Batok SMC                   1   
2                                     Bukit Batok SMC                   1   
3                                   Bukit Panjang SMC                   1   
4                                   Bukit Panjang SMC                   1   
..                                                ...                 ...   
56                                  Tanjong Pagar GRC                   5   
57                                     Ang Mo Kio GRC                   6   
58                                     Ang Mo Kio GRC                   6   
59                              Pasir Ris–Punggol GRC                   6   
60                              Pasir Ris–Punggol GRC                   6   

                                                 \
    Voters                                Party   
0    27077                People's Action Party   
1    27077           Singapore Democratic Party   
2    27077  Independent (Loses $14,500 deposit)   
3    34317                People's Action Party   
4    34317           Singapore Democratic Party   
..     ...                                  ...   
56  130752                   Singaporeans First   
57  187771                People's Action Party   
58  187771                         Reform Party   
59  187396                People's Action Party   
60  187396        Singapore Democratic Alliance   

                                                                            \
                                         Candidate(s)   Votes      Votes %   
0                                           David Ong   18234  73.02 / 100   
1                                   Sadasivam Veriyah    6588  26.38 / 100   
2                                    Samir Salim Neji     150   0.60 / 100   
3                                          Teo Ho Pin   21954  68.38 / 100   
4                                      Khung Wai Yeen   10152  31.62 / 100   
..                                                ...     ...          ...   
56  Tan Jee Say Ang Yong Guan Chirag Praful Desai ...   25998  22.29 / 100   
57  Lee Hsien Loong Ang Hin Kee Darryl David Gan T...  135316  78.64 / 100   
58  Gilbert Goh Jesse Loo Roy Ngerng Osman Sulaima...   36758  21.36 / 100   
59  Teo Chee Hean Janil Puthucheary Ng Chee Meng S...  125166  72.89 / 100   
60  Abu Mohamed Harminder Pal Singh Desmond Lim Ar...   46550  27.11 / 100   

                                
   Sample counts Swing Margins  
0       74 / 100   NaN  46.64%  
1       26 / 100   NaN  46.64%  
2        0 / 100   NaN  46.64%  
3       69 / 100  2.11  36.76%  
4       31 / 100  2.11  36.76%  
..           ...   ...     ...  
56      22 / 100   NaN  55.42%  
57      78 / 100  9.31  57.28%  
58      22 / 100  9.31  57.28%  
59      73 / 100  8.10  45.78%  
60      27 / 100  8.10  45.78%  

[61 rows x 10 columns]

In [940]:
# Rename columns as before
table_tmp.columns = ['Area', 'Seat', 'Voters', 'Party', 'Candidates', 'Votes', 'Percentage', 'Sample Counts %', 'Swing', 'Margins']

# Clean up data
table_tmp['Area'] = table_tmp['Area'].apply(lambda x: re.sub('GRC|SMC', '', x).rstrip())
table_tmp['Percentage'] = table_tmp['Percentage'].apply(lambda x: re.sub('/.*$', '', x).strip())
table_tmp['Sample Counts %'] = table_tmp['Sample Counts %'].apply(lambda x: re.sub('/.*$', '', x).strip())
table_tmp['Margins'] = table_tmp['Margins'].apply(lambda x: re.sub('%', '', x).strip())

# Convert to numeric
table_tmp['Percentage'] = table_tmp['Percentage'].astype(float)
table_tmp['Margins'] = table_tmp['Margins'].astype(float)
table_tmp['Sample Counts %'] = table_tmp['Sample Counts %'].astype(int)

# Drop 'Candidates' column
table_tmp = table_tmp.drop('Candidates', axis=1)

# Add 'GRC/SMC' column
table_tmp['GRC/SMC'] = table_tmp['Seat'].apply(lambda x: 'SMC' if x==1 else 'GRC')
# Add 'Year'
table_tmp['Year'] = 2015

table_tmp

,Area,Seat,Voters,Party,Votes,Percentage,Sample Counts %,Swing,Margins,GRC/SMC,Year
0,Bukit Batok,1,27077,People's Action Party,18234,73.02,74,NaN,46.64,SMC,2015
1,Bukit Batok,1,27077,Singapore Democratic Party,6588,26.38,26,NaN,46.64,SMC,2015
2,Bukit Batok,1,27077,"Independent (Loses $14,500 deposit)",150,0.60,0,NaN,46.64,SMC,2015
3,Bukit Panjang,1,34317,People's Action Party,21954,68.38,69,2.11,36.76,SMC,2015
4,Bukit Panjang,1,34317,Singapore Democratic Party,10152,31.62,31,2.11,36.76,SMC,2015
...,...,...,...,...,...,...,...,...,...,...,...
56,Tanjong Pagar,5,130752,Singaporeans First,25998,22.29,22,NaN,55.42,GRC,2015
57,Ang Mo Kio,6,187771,People's Action Party,135316,78.64,78,9.31,57.28,GRC,2015
58,Ang Mo Kio,6,187771,Reform Party,36758,21.36,22,9.31,57.28,GRC,2015
59,Pasir Ris–Punggol,6,187396,People's Action Party,125166,72.89,73,8.10,45.78,GRC,2015


In [941]:
# Combining with 2020's ge_results_table
ge_results_table = pd.concat([ge_results_table, table_tmp], axis=0)
ge_results_table

,Area,Seat,Voters,Party,Votes,Percentage,Sample Counts %,Swing,Margins,GRC/SMC,Year
0,Bukit Batok,1,29948,People's Action Party,15500,54.80,57,6.43,9.60,SMC,2020
1,Bukit Batok,1,29948,Singapore Democratic Party,12787,45.20,43,6.43,9.60,SMC,2020
2,Bukit Panjang,1,35437,People's Action Party,18085,53.73,56,14.65,7.46,SMC,2020
3,Bukit Panjang,1,35437,Singapore Democratic Party,15576,46.27,44,14.65,7.46,SMC,2020
4,Hong Kah North,1,28046,People's Action Party,16347,60.99,63,13.77,21.98,SMC,2020
...,...,...,...,...,...,...,...,...,...,...,...
56,Tanjong Pagar,5,130752,Singaporeans First,25998,22.29,22,NaN,55.42,GRC,2015
57,Ang Mo Kio,6,187771,People's Action Party,135316,78.64,78,9.31,57.28,GRC,2015
58,Ang Mo Kio,6,187771,Reform Party,36758,21.36,22,9.31,57.28,GRC,2015
59,Pasir Ris–Punggol,6,187396,People's Action Party,125166,72.89,73,8.10,45.78,GRC,2015


In [943]:
# Moving onto last table for ge2015
table_tmp = seats[1]
table_tmp

,Unnamed: 0,2011,2015
0,Seats,87,89
1,Electoral divisions,27,29
2,Group representation constituencies,15,16
3,Four-Member GRCs,2,6
4,Five-Member GRCs,11,8
5,Six-Member GRCs,2,2
6,Single member constituencies,12,13
7,Voters,2347198,2458058
8,Voters (overseas votes inclusive),2350873,2462926


In [944]:
# Insert column from '2011' onto electoral_divisions_table
electoral_divisions_table['2011'] = table_tmp['2011']
electoral_divisions_table

,Unnamed: 0,2015,2020,2011
0,Seats,89,93,87
1,Electoral divisions,29,31,27
2,Group representation constituencies,16,17,15
3,Four-Member GRCs,6,6,2
4,Five-Member GRCs,8,11,11
5,Six-Member GRCs,2,0,2
6,Single member constituencies,13,14,12
7,Voters,2458058,2647372,2347198
8,Voters (overseas votes inclusive),2462926,2653942,2350873


In [945]:
# Send HTTP GET request to website to retrieve 2011's election data
page = requests.get("https://en.wikipedia.org/wiki/2011_Singaporean_general_election")
page.status_code

200

In [946]:
soup = BeautifulSoup(page.content, "html.parser")
sortable_tables = soup.find_all('table', {'class': 'wikitable'})

seats = pd.read_html(str(sortable_tables), match='Seats')
table_tmp = seats[0]

In [947]:
# Inserting '2006' column to electoral_divisions_table
electoral_divisions_table['2006'] = table_tmp['2006']
electoral_divisions_table

,Unnamed: 0,2015,2020,2011,2006
0,Seats,89,93,87,84
1,Electoral divisions,29,31,27,23
2,Group representation constituencies,16,17,15,14
3,Four-Member GRCs,6,6,2,0
4,Five-Member GRCs,8,11,11,9
5,Six-Member GRCs,2,0,2,5
6,Single member constituencies,13,14,12,9
7,Voters,2458058,2647372,2347198,2158704
8,Voters (overseas votes inclusive),2462926,2653942,2350873,2159721


In [948]:
# Moving onto next table
table_tmp = seats[1]
table_tmp

,Party,Party.1,Votes,%,Seats,+/–
0,NaN,People's Action Party,1212154,60.14,81,–1
1,NaN,Workers' Party,258510,12.83,6,+5
2,NaN,National Solidarity Party,242682,12.04,0,New
3,NaN,Singapore Democratic Party,97369,4.83,0,0
4,NaN,Reform Party,86294,4.28,0,New
5,NaN,Singapore People's Party,62639,3.11,0,New
6,NaN,Singapore Democratic Alliance,55988,2.78,0,–1
7,Total,Total,2015636,100.00,87,+3
8,NaN,NaN,NaN,NaN,NaN,NaN
9,Valid votes,Valid votes,2015636,97.83,NaN,NaN


In [949]:
# Keep only the rows we're interested in to create a new temporary table as before
votes_dict = {'Year': 2011,
              'Registered Voters/Turnout': table_tmp['Votes'].iloc[12],
              'Registered Voters/Turnout %': table_tmp['%'].iloc[12],
              'Total Votes': table_tmp['Votes'].iloc[11],
              'Total Votes %': table_tmp['%'].iloc[11],
              'Valid Votes': table_tmp['Votes'].iloc[9],
              'Valid Votes %': table_tmp['%'].iloc[9],
              'Invalid/Blank Votes': table_tmp['Votes'].iloc[10],
              'Invalid/Blank Votes %': table_tmp['%'].iloc[10]
             }

votes_table_tmp = pd.DataFrame(data=votes_dict, index = [0])
votes_table_tmp.set_index('Year', inplace=True)
votes_table_tmp

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2011,2350873,93.18,2060373,100.00,2015636,97.83,44737,2.17


In [950]:
# Combine to votes_table
votes_table = pd.concat([votes_table, votes_table_tmp], axis=0)
votes_table

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2020,2651435,95.81,2540359,100.00,2494537,98.20,45822,1.80
2015,2462926,93.70,2307746,100.00,2260379,97.95,47367,2.05
2011,2350873,93.18,2060373,100.00,2015636,97.83,44737,2.17


In [951]:
# Last table for ge2011
table_tmp = seats[2]
table_tmp

Results of 2011 Singapore general election[47]                \
                                         Division Seats  Voters   
0                                    Aljunied GRC     5  143148   
1                                    Aljunied GRC     5  143148   
2                                  Ang Mo Kio GRC     6  179071   
3                                  Ang Mo Kio GRC     6  179071   
4                            Bishan–Toa Payoh GRC     5  122492   
5                            Bishan–Toa Payoh GRC     5  122492   
6                               Chua Chu Kang GRC     5  158648   
7                               Chua Chu Kang GRC     5  158648   
8                                  East Coast GRC     5  120324   
9                                  East Coast GRC     5  120324   
10                        Holland–Bukit Timah GRC     4   91607   
11                        Holland–Bukit Timah GRC     4   91607   
12                                     Jurong GRC     5  125276   
13                                     Jurong GRC     5  125276   
14                              Marine Parade GRC     5  154451   
15                              Marine Parade GRC     5  154451   
16                           Moulmein–Kallang GRC     4   87595   
17                           Moulmein–Kallang GRC     4   87595   
18                                   Nee Soon GRC     5  148290   
19                                   Nee Soon GRC     5  148290   
20                          Pasir Ris–Punggol GRC     6  168971   
21                          Pasir Ris–Punggol GRC     6  168971   
22                                  Sembawang GRC     5  142459   
23                                  Sembawang GRC     5  142459   
24                                   Tampines GRC     5  137532   
25                                   Tampines GRC     5  137532   
26                              Tanjong Pagar GRC     5  139771   
27                                 West Coast GRC     5  121045   
28                                 West Coast GRC     5  121045   
29                              Bukit Panjang SMC     1   33053   
30                              Bukit Panjang SMC     1   33053   
31                             Hong Kah North SMC     1   27701   
32                             Hong Kah North SMC     1   27701   
33                                    Hougang SMC     1   24560   
34                                    Hougang SMC     1   24560   
35                                  Joo Chiat SMC     1   22069   
36                                  Joo Chiat SMC     1   22069   
37                                Mountbatten SMC     1   23731   
38                                Mountbatten SMC     1   23731   
39                                    Pioneer SMC     1   25745   
40                                    Pioneer SMC     1   25745   
41                               Potong Pasir SMC     1   17327   
42                               Potong Pasir SMC     1   17327   
43                               Punggol East SMC     1   33281   
44                               Punggol East SMC     1   33281   
45                               Punggol East SMC     1   33281   
46                                  Radin Mas SMC     1   31014   
47                                  Radin Mas SMC     1   31014   
48                              Sengkang West SMC     1   26882   
49                              Sengkang West SMC     1   26882   
50                                    Whampoa SMC     1   21622   
51                                    Whampoa SMC     1   21622   
52                                      Yuhua SMC     1   23195   
53                                      Yuhua SMC     1   23195   

                                                       \
                                                Party   
0                                      Workers' Party   
1                               People's Action Party   
2                     

In [952]:
# Rename columns again
table_tmp.columns = ['Area', 'Seat', 'Voters', 'Party', 'Candidates', 'Votes', 'Percentage', 'Swing', 'Margins']

# Clean up data
table_tmp['Area'] = table_tmp['Area'].apply(lambda x: re.sub('GRC|SMC', '', x).rstrip())
table_tmp['Percentage'] = table_tmp['Percentage'].apply(lambda x: re.sub('/.*$', '', x).strip())
table_tmp['Margins'] = table_tmp['Margins'].apply(lambda x: re.sub('%', '', x).strip())

# 2011 and earlier elections had Walkovers. Converting them to null values
table_tmp['Votes'] = table_tmp['Votes'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)
table_tmp['Percentage'] = table_tmp['Percentage'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)
table_tmp['Swing'] = table_tmp['Swing'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)
table_tmp['Margins'] = table_tmp['Margins'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)

# Converting to numeric
table_tmp['Percentage'] = table_tmp['Percentage'].astype(float)
table_tmp['Margins'] = table_tmp['Margins'].astype(float)

table_tmp.dtypes

Area           object
Seat            int64
Voters          int64
Party          object
Candidates     object
Votes          object
Percentage    float64
Swing          object
Margins       float64
dtype: object

In [953]:
# Dropping 'Candidates' column
table_tmp = table_tmp.drop('Candidates', axis=1)

# Adding 'GRC/SMC' column
table_tmp['GRC/SMC'] = table_tmp['Seat'].apply(lambda x: 'SMC' if x==1 else 'GRC')
# Adding 'Year'
table_tmp['Year'] = 2011

table_tmp

,Area,Seat,Voters,Party,Votes,Percentage,Swing,Margins,GRC/SMC,Year
0,Aljunied,5,143148,Workers' Party,72289,54.72,10.81,9.44,GRC,2011
1,Aljunied,5,143148,People's Action Party,59829,45.28,10.81,9.44,GRC,2011
2,Ang Mo Kio,6,179071,People's Action Party,112677,69.33,3.19,38.66,GRC,2011
3,Ang Mo Kio,6,179071,Reform Party,49851,30.67,3.19,38.66,GRC,2011
4,Bishan–Toa Payoh,5,122492,People's Action Party,62385,56.93,NaN,13.86,GRC,2011
5,Bishan–Toa Payoh,5,122492,Singapore People's Party,47205,43.07,NaN,13.86,GRC,2011
6,Chua Chu Kang,5,158648,People's Action Party,89710,61.20,0.83,22.40,GRC,2011
7,Chua Chu Kang,5,158648,National Solidarity Party,56885,38.80,NaN,22.40,GRC,2011
8,East Coast,5,120324,People's Action Party,59992,54.83,9.03,9.66,GRC,2011
9,East Coast,5,120324,Workers' Party,49429,45.17,9.03,9.66,GRC,2011


In [954]:
# Combine with ge_results_table
ge_results_table = pd.concat([ge_results_table, table_tmp], axis=0)
ge_results_table

,Area,Seat,Voters,Party,Votes,Percentage,Sample Counts %,Swing,Margins,GRC/SMC,Year
0,Bukit Batok,1,29948,People's Action Party,15500,54.80,57.0,6.43,9.60,SMC,2020
1,Bukit Batok,1,29948,Singapore Democratic Party,12787,45.20,43.0,6.43,9.60,SMC,2020
2,Bukit Panjang,1,35437,People's Action Party,18085,53.73,56.0,14.65,7.46,SMC,2020
3,Bukit Panjang,1,35437,Singapore Democratic Party,15576,46.27,44.0,14.65,7.46,SMC,2020
4,Hong Kah North,1,28046,People's Action Party,16347,60.99,63.0,13.77,21.98,SMC,2020
...,...,...,...,...,...,...,...,...,...,...,...
49,Sengkang West,1,26882,Workers' Party,10591,41.89,NaN,NaN,16.22,SMC,2011
50,Whampoa,1,21622,People's Action Party,13028,66.10,NaN,NaN,32.20,SMC,2011
51,Whampoa,1,21622,National Solidarity Party,6680,33.90,NaN,NaN,32.20,SMC,2011
52,Yuhua,1,23195,People's Action Party,14093,66.86,NaN,NaN,33.72,SMC,2011


In [955]:
# Send HTTP GET request to website to retrieve 2006's data
page = requests.get("https://en.wikipedia.org/wiki/2006_Singaporean_general_election")
page.status_code

200

In [956]:
soup = BeautifulSoup(page.content, "html.parser")
sortable_tables = soup.find_all('table', {'class': 'wikitable'})

# Get tables with the word 'Seats' in it
seats = pd.read_html(str(sortable_tables), match='Seats')
table_tmp = seats[0]
table_tmp

,Party,Party.1,Votes,%,+/–,Seats,+/–.1
0,NaN,People's Action Party,748130,66.60,–8.69,82,0
1,NaN,Workers' Party,183578,16.34,+13.29,1,0
2,NaN,Singapore Democratic Alliance,145628,12.96,+0.93,1,0
3,NaN,Singapore Democratic Party,45937,4.09,–4.00,0,0
4,Total,Total,1123273,100.00,–,84,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Valid votes,Valid votes,1123273,97.68,NaN,NaN,NaN
7,Invalid/blank votes,Invalid/blank votes,26730,2.32,NaN,NaN,NaN
8,Total votes,Total votes,1150003,100.00,NaN,NaN,NaN
9,Registered voters/turnout,Registered voters/turnout,2159721,94.00,NaN,NaN,NaN


In [957]:
# Keep rows we want to create a new temporary table
votes_dict = {'Year': 2006,
              'Registered Voters/Turnout': table_tmp['Votes'].iloc[9],
              'Registered Voters/Turnout %': table_tmp['%'].iloc[9],
              'Total Votes': table_tmp['Votes'].iloc[8],
              'Total Votes %': table_tmp['%'].iloc[8],
              'Valid Votes': table_tmp['Votes'].iloc[6],
              'Valid Votes %': table_tmp['%'].iloc[6],
              'Invalid/Blank Votes': table_tmp['Votes'].iloc[7],
              'Invalid/Blank Votes %': table_tmp['%'].iloc[7]
             }

votes_table_tmp = pd.DataFrame(data=votes_dict, index = [0])
votes_table_tmp.set_index('Year', inplace=True)
votes_table_tmp

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2006,2159721,94.00,1150003,100.00,1123273,97.68,26730,2.32


In [958]:
# Combine to votes_table
votes_table = pd.concat([votes_table, votes_table_tmp], axis=0)
votes_table

,Registered Voters/Turnout,Registered Voters/Turnout %,Total Votes,Total Votes %,Valid Votes,Valid Votes %,Invalid/Blank Votes,Invalid/Blank Votes %
Year,,,,,,,,
2020,2651435,95.81,2540359,100.00,2494537,98.20,45822,1.80
2015,2462926,93.70,2307746,100.00,2260379,97.95,47367,2.05
2011,2350873,93.18,2060373,100.00,2015636,97.83,44737,2.17
2006,2159721,94.00,1150003,100.00,1123273,97.68,26730,2.32


In [959]:
# Last table for 2006
table_tmp = seats[1]

# There are both rows and columns with all null values. Dropping those
table_tmp.dropna(axis=0, how='all', inplace=True)
table_tmp.dropna(axis=1, how='all', inplace=True)

# Renaming columns
table_tmp.columns = ['Area', 'Seat', 'Voters', 'Party', 'Candidates', 'Votes', 'Percentage']

# Removing a couple of rows that are actually headers
table_tmp.drop(table_tmp[table_tmp['Seat'] == 'Group Representation Constituency (GRC) (75 seats)'].index, inplace=True)
table_tmp.drop(table_tmp[table_tmp['Seat'] == 'Single Member Constituency (SMC) (9 seats)'].index, inplace=True)

# Resetting the index
table_tmp.reset_index(drop = True, inplace=True)
table_tmp

,Area,Seat,Voters,Party,Candidates,Votes,Percentage
0,Bukit Panjang SMC,1,30452,People's Action Party,Teo Ho Pin,21652,77.18 / 100
1,Bukit Panjang SMC,1,30452,Singapore Democratic Party,Ling How Doong,6400,22.82 / 100
2,Chua Chu Kang SMC,1,24975,People's Action Party,Gan Kim Yong,14156,60.37 / 100
3,Chua Chu Kang SMC,1,24975,Singapore Democratic Alliance,Steve Chia,9292,39.63 / 100
4,Hougang SMC,1,23759,Workers' Party,Low Thia Khiang,13989,62.74 / 100
5,Hougang SMC,1,23759,People's Action Party,Eric Low,8308,37.26 / 100
6,Joo Chiat SMC,1,21858,People's Action Party,Chan Soo Sen,12226,65.01 / 100
7,Joo Chiat SMC,1,21858,Workers' Party,Tan Bin Seng,6580,34.99 / 100
8,MacPherson SMC,1,21041,People's Action Party,Matthias Yao,13184,68.28 / 100
9,MacPherson SMC,1,21041,Singapore Democratic Alliance,Sin Kek Tong,6067,31.52 / 100


In [960]:
# Clean up data
table_tmp['Area'] = table_tmp['Area'].apply(lambda x: re.sub('GRC|SMC', '', x).rstrip())
table_tmp['Percentage'] = table_tmp['Percentage'].apply(lambda x: re.sub('/.*$', '', x).strip())

# Convert 'Uncontested Walkover' to null values
table_tmp['Votes'] = table_tmp['Votes'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)
table_tmp['Percentage'] = table_tmp['Percentage'].apply(lambda x: np.nan if x == 'Uncontested Walkover' else x)

# Converting to numeric
table_tmp['Percentage'] = table_tmp['Percentage'].astype(float)
table_tmp['Seat'] = table_tmp['Seat'].astype(int)
table_tmp['Voters'] = table_tmp['Voters'].astype(int)
# Special case as this contains null values
table_tmp['Votes'] = table_tmp['Votes'].astype('Int32')

table_tmp.dtypes

Area           object
Seat            int32
Voters          int32
Party          object
Candidates     object
Votes           Int32
Percentage    float64
dtype: object

In [961]:
table_tmp

,Area,Seat,Voters,Party,Candidates,Votes,Percentage
0,Bukit Panjang,1,30452,People's Action Party,Teo Ho Pin,21652,77.18
1,Bukit Panjang,1,30452,Singapore Democratic Party,Ling How Doong,6400,22.82
2,Chua Chu Kang,1,24975,People's Action Party,Gan Kim Yong,14156,60.37
3,Chua Chu Kang,1,24975,Singapore Democratic Alliance,Steve Chia,9292,39.63
4,Hougang,1,23759,Workers' Party,Low Thia Khiang,13989,62.74
5,Hougang,1,23759,People's Action Party,Eric Low,8308,37.26
6,Joo Chiat,1,21858,People's Action Party,Chan Soo Sen,12226,65.01
7,Joo Chiat,1,21858,Workers' Party,Tan Bin Seng,6580,34.99
8,MacPherson,1,21041,People's Action Party,Matthias Yao,13184,68.28
9,MacPherson,1,21041,Singapore Democratic Alliance,Sin Kek Tong,6067,31.52


In [962]:
# Drop 'Candidates' column
table_tmp = table_tmp.drop('Candidates', axis=1)

# Add 'GRC/SMC' column
table_tmp['GRC/SMC'] = table_tmp['Seat'].apply(lambda x: 'SMC' if x==1 else 'GRC')

# Add 'Year' column
table_tmp['Year'] = 2006

table_tmp

,Area,Seat,Voters,Party,Votes,Percentage,GRC/SMC,Year
0,Bukit Panjang,1,30452,People's Action Party,21652,77.18,SMC,2006
1,Bukit Panjang,1,30452,Singapore Democratic Party,6400,22.82,SMC,2006
2,Chua Chu Kang,1,24975,People's Action Party,14156,60.37,SMC,2006
3,Chua Chu Kang,1,24975,Singapore Democratic Alliance,9292,39.63,SMC,2006
4,Hougang,1,23759,Workers' Party,13989,62.74,SMC,2006
5,Hougang,1,23759,People's Action Party,8308,37.26,SMC,2006
6,Joo Chiat,1,21858,People's Action Party,12226,65.01,SMC,2006
7,Joo Chiat,1,21858,Workers' Party,6580,34.99,SMC,2006
8,MacPherson,1,21041,People's Action Party,13184,68.28,SMC,2006
9,MacPherson,1,21041,Singapore Democratic Alliance,6067,31.52,SMC,2006


In [963]:
# Combine with ge_results_table
ge_results_table = pd.concat([ge_results_table, table_tmp], axis=0)
ge_results_table

,Area,Seat,Voters,Party,Votes,Percentage,Sample Counts %,Swing,Margins,GRC/SMC,Year
0,Bukit Batok,1,29948,People's Action Party,15500,54.80,57.0,6.43,9.60,SMC,2020
1,Bukit Batok,1,29948,Singapore Democratic Party,12787,45.20,43.0,6.43,9.60,SMC,2020
2,Bukit Panjang,1,35437,People's Action Party,18085,53.73,56.0,14.65,7.46,SMC,2020
3,Bukit Panjang,1,35437,Singapore Democratic Party,15576,46.27,44.0,14.65,7.46,SMC,2020
4,Hong Kah North,1,28046,People's Action Party,16347,60.99,63.0,13.77,21.98,SMC,2020
...,...,...,...,...,...,...,...,...,...,...,...
34,Sembawang,6,184804,People's Action Party,130170,76.70,NaN,NaN,NaN,GRC,2006
35,Sembawang,6,184804,Singapore Democratic Party,39537,23.30,NaN,NaN,NaN,GRC,2006
36,Marine Parade,6,155149,People's Action Party,<NA>,NaN,NaN,NaN,NaN,GRC,2006
37,Ang Mo Kio,6,159872,People's Action Party,96636,66.14,NaN,NaN,NaN,GRC,2006


In [965]:
# Reorder electoral_divisions_table to fix messy order
electoral_divisions_table = electoral_divisions_table.loc[:, ['Unnamed: 0', '2006', '2011', '2015', '2020']]
electoral_divisions_table

,Unnamed: 0,2006,2011,2015,2020
0,Seats,84,87,89,93
1,Electoral divisions,23,27,29,31
2,Group representation constituencies,14,15,16,17
3,Four-Member GRCs,0,2,6,6
4,Five-Member GRCs,9,11,8,11
5,Six-Member GRCs,5,2,2,0
6,Single member constituencies,9,12,13,14
7,Voters,2158704,2347198,2458058,2647372
8,Voters (overseas votes inclusive),2159721,2350873,2462926,2653942


In [966]:
import os
# Set output directory
output_dir = os.path.join('..', 'data')
os.makedirs(output_dir, exist_ok=True)

# Export data to CSV files
output_file = os.path.join(output_dir, 'wiki_ge_results.csv')
ge_results_table.to_csv(output_file, index=False)

output_file = os.path.join(output_dir, 'wiki_electoral_divisions.csv')
electoral_divisions_table.to_csv(output_file, index=False)

output_file = os.path.join(output_dir, 'wiki_votes.csv')
votes_table.to_csv(output_file, index=True)